In [1]:
import numpy as np
import random
import json
import pandas as pd

In [2]:
queries = ["SELECT","INSERT","UPDATE","DELETE","ALTER"]
roles = ["Administrator","Project Manager","Team Lead","Developer","Quality Assurance"]
tables = ["users","permissions","employee","projects","project_tasks","bug_logs","test_cases","code_repo"]

In [3]:
def read_schema_info():
    
    with open("schema.json","r") as json_file:
        schema = json.load(json_file)
    
    with open("permissions.json", "r") as json_file:
        permissions = json.load(json_file)
    
    return schema, permissions

In [4]:
def check_access(role,permissions,table_name,query_type):
    ind=0
    if query_type=="SELECT":
        ind=0
    elif query_type=="INSERT" or query_type=="UPDATE":
        ind=1
    else:
        ind=2
        
    return permissions[role][table_name][ind]

In [5]:
def get_avg_size(tables_selected,schema):
    avg_size=0.0
    total_size=1.0
    
    percent=random.uniform(0.01,1)
    lower_limit=percent-0.05*percent; lower_limit=max(0.01,lower_limit)
    upper_limit=percent+0.05*percent; upper_limit=min(1,upper_limit)
    
    for table in tables_selected:
        size = schema[table]["num_rows"]
        total_size += size
        m=random.random()
        if(m<0.15):
            avg_size+=size*random.uniform(0.01,lower_limit)
        elif(m>0.85):
            avg_size+=size*random.uniform(upper_limit,1)
        else:
            avg_size+=size*random.uniform(lower_limit,upper_limit)
    
    return avg_size/total_size

In [6]:
def assign_role_score(role):
    if role=="Administrator":
        return 1.0
    elif role=="Project Manager":
        return 0.8
    elif role=="Team Lead":
        return 0.7
    elif role=="Developer":
        return 0.6
    else:
        return 0.5

In [15]:
queries = []
role_column = []
table_scores = []
avg_data_sizes = []
frequency_scores = []

In [16]:
def generate_data(query_type):
    rows=0
    
    schema, permissions = read_schema_info()
    
    query_code=0
    
    if query_type == "SELECT":
        rows = 600
        query_code = 1
    else:
        rows = 100
        query_code = 2
    
    for i in range(rows):
        
        role = random.choice(roles)
        
        role_score = assign_role_score(role)
        
        table = random.choice(tables)
        
        table_count = 1
        
        table_score = 0
        
        tables_selected = []
        
        access = False
        
        while access==False:
            if check_access(role,permissions,table,query_type):
                break
                    
            table = random.choice(tables)
            
        if query_type == "SELECT":
            table_mappings = schema[table]["table_mappings"]
            
            allowed_tables = [table]
            
            for table_mapping in table_mappings:
                if check_access(role,permissions,table_mapping,query_type):
                    table_count += 1
                    allowed_tables.append(table_mapping)
            
            tab = random.randint(1,table_count)
            
            while len(tables_selected) < table_count:
                t = random.choice(allowed_tables)
                if t not in tables_selected:
                    tables_selected.append(t)
            
            for tb in tables_selected:
                table_score += schema[tb]["sensitivity"]
        
        else:
            tables_selected.append(table)
            table_score += schema[table]["sensitivity"]
        
        avg_data_size = get_avg_size(tables_selected,schema)
        
        frequency_score = random.uniform(0.00001,14.0165348)
        
        queries.append(query_code)
        role_column.append(role_score)
        table_scores.append(table_score)
        avg_data_sizes.append(avg_data_size)
        frequency_scores.append(frequency_score)

In [17]:
generate_data("SELECT")
generate_data("INSERT")
generate_data("UPDATE")
generate_data("DELETE")

In [18]:
df = pd.DataFrame()
df["Query"] = queries
df["Role"] = role_column
df["Table Sensitivity"] = table_scores
df["Average Data Size"] = avg_data_sizes
df["Frequency Score"] = frequency_scores

In [19]:
df.to_csv('output.csv', index=False)